In [3]:
import re
import numpy as np
import multiprocessing as mp
import sys
import pandas as pd
from xtquant import xtdata
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
from cook_data import cook_data
from read_data import get_market_data,get_main_contract_code

In [2]:
from xtquant import xtdatacenter as xtdc  
xtdc.set_token('4065054877ce5724155dbc5bcba200381ce5eb35')
xtdc.init()

In [4]:
    continuous_contract_code = 'TA00.ZF'  # 示例期货代码
    start_date = '20250604'
    end_date = '20250613'

In [5]:
code = get_main_contract_code(continuous_contract_code, start_time=start_date, end_time=end_date)

***** xtdata连接成功 *****
服务信息: {'tag': 'xtquant', 'version': '1.0'}
服务地址: 127.0.0.1:58609
数据路径: c:\Users\HP\Desktop\project\branches\Liang\work_space-main\data\datadir
设置xtdata.enable_hello = False可隐藏此消息



In [6]:
code

'TA509.ZF'

In [8]:
future_dict = get_market_data(codes=[code], period='1m', start_time=start_date, end_time=end_date, count=-1)

In [ ]:
future_dict

In [10]:
future_df = cook_data(raw_data=future_dict[code])[['date', 'close', 'trading_date']].copy()

日期超出交易日历范围


UnboundLocalError: cannot access local variable 'trading_date' where it is not associated with a value

In [5]:
xtdata.get_option_detail_data('SH509C2840.ZF')

{'ExchangeID': 'CZCE',
 'InstrumentID': 'SH509C2840',
 'InstrumentName': 'SH509C2840',
 'ProductID': 'SHC',
 'ProductType': 2,
 'OpenDate': '20250102',
 'CreateDate': '20250102',
 'ExpireDate': '20250813',
 'PreClose': 6.5,
 'SettlementPrice': 6.5,
 'UpStopPrice': 169.5,
 'DownStopPrice': 0.5,
 'LongMarginRatio': 0.0,
 'ShortMarginRatio': 0.0,
 'PriceTick': 0.5,
 'VolumeMultiple': 30,
 'MaxMarketOrderVolume': 2,
 'MinMarketOrderVolume': 1,
 'MaxLimitOrderVolume': 100,
 'MinLimitOrderVolume': 1,
 'OptUnit': 30.0,
 'MarginUnit': 1.7976931348623157e+308,
 'OptUndlCode': 'SH2509',
 'OptUndlMarket': 'ZF',
 'OptUndlCodeFull': 'SH2509.ZF',
 'OptExercisePrice': 2840.0,
 'NeeqExeType': 2147483647,
 'OptUndlRiskFreeRate': 0.0165,
 'OptUndlHistoryRate': 0.1962,
 'EndDelivDate': '20250813',
 'optType': 'CALL',
 'ProductCode': 'SH.ZF'}

In [25]:
A = get_market_data(codes=['SH509C2840.ZF'], period='1m', start_time='20250401100400', end_time='20250401100400', count=-1)

In [26]:
A = A['SH509C2840.ZF']

In [27]:
A

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250401100400,1743473040000,67.0,67.0,67.0,67.0,0,0.0,0.0,86,67.0,0


In [10]:
if A['SH2509C2840.ZF'].empty:
    print('1')

1


In [8]:
option_list = xtdata.get_option_list(undl_code='SH2409.ZF',dedate='',opttype='PUT')

In [ ]:
option_list

In [4]:
#['date','close','trading_date'].copy()

In [9]:
def strategy(code: str, start_date: str, end_date: str, max_signals: int = 30, lookback_days: int = 20, k: float = 0.08, close_n: int = 10) -> pd.DataFrame:
    #max_signals:该日积累该次数信号开仓，lookback_days:回溯窗口期大小，k：ratio of option's K and future price, close_n:于行权日前n天平仓
    future_dict = get_market_data(codes=[code], period='1m', start_time=start_date, end_time=end_date, count=-1)
    future_df = cook_data(raw_data=future_dict[code])[['date', 'close', 'trading_date']].copy()
    
    signal_count_pos = 0
    signal_count_neg = 0
    
    # 初始化列
    future_df['signal_type'] = 0
    future_df['close_min'] = 0.0
    future_df['close_max'] = 0.0
    
    price_dict = {}
    rows_to_drop = []
    #print("开始遍历数据...")
    
    for i in range(len(future_df)):
        count = 0
        current_time = future_df.iloc[i]['date']
        current_date = current_time.date()
        if count == 0:
            yesterday = current_date
            count += 1

        current_close = future_df.iloc[i]['close']
        
        current_trading_date = future_df.iloc[i]['trading_date']

        close_min = None
        close_max = None
        
        cache_key = current_trading_date
        
        if cache_key in price_dict:
            close_min = price_dict[cache_key]['close_min']
            close_max = price_dict[cache_key]['close_max']
            future_df.at[i, 'close_min'] = close_min
            future_df.at[i, 'close_max'] = close_max
        else:
            try:
                dict_data = get_market_data([code], '1d', '', current_trading_date, lookback_days)
                
                if code in dict_data and not dict_data[code].empty:
                    close_min = dict_data[code]['close'].min()
                    close_max = dict_data[code]['close'].max()
                    future_df.at[i, 'close_min'] = close_min
                    future_df.at[i, 'close_max'] = close_max
                    price_dict[cache_key] = {'close_min': close_min, 'close_max': close_max}
                else:
                    print(f"无法获取交易时间 {current_time} 的历史数据，标记删除行 {i}")
                    rows_to_drop.append(i)
                    continue
                    
            except Exception as e:
                print(f"获取数据错误: {e}，标记删除行 {i}")
                rows_to_drop.append(i)
                continue
        
        if close_min is None or close_max is None:
            print(f"行 {i} 数据无效，标记删除")
            rows_to_drop.append(i)
            continue
        
        if current_close > close_max:
            signal_count_pos += 1
            signal_count_neg = 0  
            
        elif current_close < close_min:
            signal_count_neg += 1
            signal_count_pos = 0  

        if signal_count_pos >= max_signals:
            future_df.at[i, 'signal_type'] = 1
            # print(f"触发卖出PUT信号，交易时间: {current_date}, 价格: {current_close:.2f}")
            signal_count_pos = 0
            
        if signal_count_neg >= max_signals:
            future_df.at[i, 'signal_type'] = -1
            # print(f"触发卖出CALL信号，交易时间: {current_date}, 价格: {current_close:.2f}")
            signal_count_neg = 0

        if yesterday != current_date:
            signal_count_pos = 0
            signal_count_neg = 0

        yesterday = current_date
        
    if rows_to_drop:
        print(f"\n删除 {len(rows_to_drop)} 行无效数据")
        future_df = future_df.drop(rows_to_drop).reset_index(drop=True)
        print(f"删除后剩余数据: {len(future_df)} 条")
    future_df = future_df.dropna().reset_index(drop=True)
    
    print(f"\n=== 策略执行完成 ===")
    print(f"最终数据量: {len(future_df)} 条")

    return future_df

In [7]:
try:
    strategy(code='SH2509.ZF', start_date='20250501', end_date='20250601', max_signals= 30,lookback_days=1, k=0.05, close_n=5)
except:
    import traceback
    traceback.print_exc()

数据解析完成: 6453 条有效数据


Traceback (most recent call last):
  File "c:\Anaconda\envs\oye\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: ('date', 'close')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_21464\2685095480.py", line 2, in <module>
    strategy(code='SH2509.ZF', start_date='20250501', end_date='20250601', max_signals= 30,lookback_days=1, k=0.05, close_n=5)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_2

In [10]:
strategy(code='SH2509.ZF', start_date='20250501', end_date='20250601', max_signals= 30,lookback_days=1, k=0.05, close_n=5)

数据解析完成: 6453 条有效数据

=== 策略执行完成 ===
最终数据量: 6453 条


,date,close,trading_date,signal_type,close_min,close_max
0,2025-05-06 09:01:00,2479.0,20250506,0,2498.0,2498.0
1,2025-05-06 09:02:00,2475.0,20250506,0,2498.0,2498.0
2,2025-05-06 09:03:00,2471.0,20250506,0,2498.0,2498.0
3,2025-05-06 09:04:00,2476.0,20250506,0,2498.0,2498.0
4,2025-05-06 09:05:00,2478.0,20250506,0,2498.0,2498.0
...,...,...,...,...,...,...
6448,2025-05-30 14:56:00,2453.0,20250530,0,2456.0,2456.0
6449,2025-05-30 14:57:00,2455.0,20250530,0,2456.0,2456.0
6450,2025-05-30 14:58:00,2455.0,20250530,0,2456.0,2456.0
6451,2025-05-30 14:59:00,2456.0,20250530,0,2456.0,2456.0


In [42]:
strategy(code_input: str, start_date: str, end_date: str, max_signals: int = 30, lookback_days: int = 20, k: float = 0.08, close_n: int = 10)

SyntaxError: invalid syntax (1248451188.py, line 1)

In [ ]:
def get_price(future_code:str, period:int, start_time:str='', end_time:str='') -> pd.DataFrame:
    future_df = get_market_data(future_code, period, start_time, end_time)
    option_code =

In [4]:
future_df = get_market_data('SH2509.ZF', '1m', '20250501', '20250601')['close']

***** xtdata连接成功 *****
服务信息: {'tag': 'xtquant', 'version': '1.0'}
服务地址: 127.0.0.1:58609
数据路径: C:\Users\HP\Desktop\work_space-main\data\datadir
设置xtdata.enable_hello = False可隐藏此消息



20250506090100    2479.0
20250506090200    2475.0
20250506090300    2471.0
20250506090400    2476.0
20250506090500    2478.0
                   ...  
20250530145600    2453.0
20250530145700    2455.0
20250530145800    2455.0
20250530145900    2456.0
20250530150000    2456.0
Name: close, Length: 6453, dtype: float64

In [6]:
def parse_option_code(option_code):
  
    code  = option_code.split('.')[0]
    if '-' in code:
        parts = code.split('-')
        if len(parts) == 3:
            underlying = parts[0]
            option_type = 'call' if parts[1].upper() == 'C' else 'put'
            strike = int(parts[2])
        else:
            import traceback
            traceback.print_exc()

    else:

        call_pos = code.find('C') # 找不到指定字符串的时候返回 -1
        put_pos = code.find('P')

        if call_pos != -1:
            option_type = 'call'
            underlying = code[:call_pos]
            strike = int(code[call_pos + 1:])
        elif put_pos != -1:
            option_type = 'put'
            underlying = code[:put_pos]
            strike = int(code[put_pos + 1:])
        else:
            import traceback
            traceback.print_exc()

    return {
        'option_type': option_type,
        'underlying': underlying,
        'strike': strike,
        'exchange': option_code.split('.')[-1] if '.' in option_code else None
    }

In [7]:
parse_option_code('SH509C2840.ZF')

{'option_type': 'call',
 'underlying': 'SH509',
 'strike': 2840,
 'exchange': 'ZF'}